# Отчет по статье

Разберем, что происходит в [данной статье](https://towardsdatascience.com/anime-recommendation-engine-from-matrix-factorization-to-learning-to-rank-845d4a9af335).

## Данные

В начале статьи рассказывается о данных. Они взяты с [соревнования на kaggle](https://www.kaggle.com/azathoth42/myanimelist). Авторы посмотрели, как вообще выглядят данные (первый параграф статьи, а также [папка на их github](https://github.com/Preetikasri/Anime-Recommendation-Engine/tree/master/dataset_explore)), но, какую-то полезную информацию для своего [итогового движка](https://github.com/Preetikasri/Anime-Recommendation-Engine/blob/master/SRC/rank_based_algo_code.py) брать не стали (итоговая версия работает только на рейтингах, без каких-либо дополнительных признаков).

На [github](https://github.com/Preetikasri/Anime-Recommendation-Engine/tree/master/Notebook) есть пара ноутбуков с чисткой данных, но их результаты не используются в других ноутбуках -- авторы каждый раз по-своему выбирают данные, причем не всегда из одних и тех же файлов.

## Алгоритмы

В статье обсуждается 5 алгоритмов. Сначала идет три классических подхода к решению Collaborative Filtering. Затем идет два алгоритма Learn to Rank.

### ALS

ALS (Alternating Least Squares) -- итеративно ищет факторизованное представление матрицы рейтингов (R) в виде прозведения матриц U и P.

Глобальная задача -- найти минимум $J = ||R - UP^{T}||_{2} + \lambda(||U||_{2} + ||P||_{2})$. Она решается следующим итеративным алгоритмом:
1. Сначала фиксируется матрица P, и решается задача обычной линейной регрессии для U;
2. Затем фиксируется матрица U, и решается задача обычной линейной регрессии для P.

$$\forall{u_{i}}:\ J(u_{i}) = ||R_{i} - u_{i}P^{T}||_{2} + \lambda \cdot ||u_{i}||_{2}$$
$$\forall{p_{j}}:\ J(p_{j}) = ||R_{j} - Up_{j}^{T}||_{2} + \lambda \cdot ||p_{j}||_{2}$$

Решение:

$$u_{i} = (P^{T}P + \lambda I)^{-1}P^{T}R_{i}$$
$$p_{j} = (U^{T}U + \lambda I)^{-1}U^{T}R_{j}$$

### SVD

Решает глобальную задачу с помощью SVD разложения.

### NN

Архитектура сети:  
![](https://cdn-images-1.medium.com/max/1600/1*aLJrZkD0tF-TlHAfyCcBRA.png)

### Промежуточные результаты

Каждый из алгоритмов давал хорошие результаты, но еще большего успеха получилось добиться после ансамблирования всех трех алгоритмов.

![](https://cdn-images-1.medium.com/max/1600/1*xf7xhhAONUsCFjSDAZmbWg.png)

### Eigen Rank

Pipeline алгоритма Eigen Rank (на примере рекомендаций аниме):
* Для каждого пользователя сделать следующее
    * Используя корелляционный коэффициент Кендала Тау найти ближайших соседей ($N_{u}$);
    * Посчитать функцию предпочтения для каждой пары объектов.
* С помощью жадного алгоритма провести определение рангов объектов.

Функция предпочтения:  
![](https://cdn-images-1.medium.com/max/1600/1*cJJwvB814KjOJGuI2f3k3w.png)

### LambdaMART

LambdaMART является комбинацией LambdaRank и MART. LambdaRank предлагает новую функцию потерь, по которой будет производится оптимизация RankNet (стремится минимизировать количество инверсий в ранжировании; используется SGD), а MART -- использует градиентный бустинг над деревьями для решения задач предсказания. В [оригинальной статье](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf) утверждается, что каждый подход улучшает результат предыдущего (RankNet < LambdaRank < LambdaMART).

### Финальные результаты

Последние два алгоритма обучались на меньшей выборке, поэтому результаты не такие хорошие, как раньше.

|            | NDCG | MAP  |
|------------|------|------|
| Eigen Rank | 0.72 | 0.63 |
| LambdaMART | 0.60 | 0.56 |

## Метрики

Авторы сначала использовали неподходящую метрику (RMSE), поэтому в конце статьи приведен обзор популярных метрик для задач рекомендации и ранжирования.

### NDCG

NDCG (Normalized Discounted Cumulative Gain) -- показывает, как сильно текущий рейтинг отличается от идеального (т.е. правильно отсортированных данных). NDCG оприается на два предположения:
1. Высоко-релевантные документы полезнее, если появляются раньше в выдаче;
2. Высоко-релевантные документы важнее просто релевантных документов, которые в свою очередь важнее нерелевантных документов.

NDCG@K -- вычисления проводятся для первых K рекомендаций.

Формулы, по которым рассчитывается NDCG:
$$NDCG_{p} = \cfrac{DCG_{p}}{IDCG_{p}}$$
$$DCG_{p} = \sum_{i=1}^{p}\cfrac{2^{rel_{i}} - 1}{log_{2}(i + 1)}$$
$$IDCG_{p} = \sum_{i=1}^{|REL|}\cfrac{2^{rel_{i}} - 1}{log_{2}(i + 1)}$$

Можно считать DCG (и, соответственно, NDCG) по-другим формулам, но приведенные выше сильнее зависят от релевантности документов.

### MAP

MAP (Mean Average Precision) -- показывает среднюю точность ранжирования.

$$MAP = \cfrac{\sum_{q=1}^{Q}AP(q)}{Q}$$
$$AP = \cfrac{\sum_{k=1}^{n}P(k) \cdot rel(k)}{\text{number of relevant documents}}$$

Q -- число запросов; P(k) -- precision первых k документов, rel(k) -- индикатор того, релевантен ли документ с рангом k.

MAP@K -- вычисления проводятся для первых K рекомендаций.

## Заключение

Статья описывает, как и что студенты делали в качестве проекта по курсу -- поэтому в ней, как мне кажется, не хватает формальных объяснений. В целом описано много методов, о которых можно дальше искать информацию.

Репозиторий плохо организован -- разделение по папкам не помогает разбираться в коде, отсутствует README, много каких-то незапущенных ячеек, комментариев.